In [1]:
import numpy as np

# https://github.com/NickAlger/helper_functions/blob/master/unit_speed_derivatives.py
from unit_speed_derivatives import unit_speed_derivatives

In [2]:
from scipy.integrate import solve_ivp
from scipy import misc

def v_unit(t, x):
    return np.array([-x[1], x[0]])/np.linalg.norm(x) # circle


def scaling_factor(x):
    return 1.0 + (x[0] - 0.5)**2 + (x[1] + 0.3)**2 + (x[0] + 0.2)**4 + (x[1] - 0.1)**4 # Randomish positive polynomial


def v(t, x):
    return v_unit(t, x) * scaling_factor(x)


y0 = np.array([1,1])/np.sqrt(2.)

ode_tol = 5e-14
ode_method = 'RK45'
finite_difference_step_size = 5e-3
num_finite_difference_steps = 13

def y_fct0(T):
    return solve_ivp(v, (0.0, T), y0, rtol=ode_tol, atol=ode_tol, method=ode_method).y[0,-1]

def y_fct1(T):
    return solve_ivp(v, (0.0, T), y0, rtol=ode_tol, atol=ode_tol, method=ode_method).y[1,-1]

num_derivatives = 4
YY = np.zeros((2, num_derivatives))
for k in range(num_derivatives):
    YY[0,k] = misc.derivative(y_fct0, 0.0, dx=finite_difference_step_size, n=k + 1, order=num_finite_difference_steps)
    YY[1,k] = misc.derivative(y_fct1, 0.0, dx=finite_difference_step_size, n=k + 1, order=num_finite_difference_steps)

XX = unit_speed_derivatives(YY)

XX_true = (1. / np.sqrt(2.)) * np.array([[-1, -1,  1, 1, -1, -1,  1, 1, -1, -1,  1, 1],
                                         [ 1, -1, -1, 1,  1, -1, -1, 1,  1, -1, -1, 1]])
XX_true = XX_true[:,:num_derivatives]

err = np.linalg.norm(XX - XX_true)
print('err=', err)

err= 6.500262615625636e-08
